# Yeast Polarization Example

This is a model of spontations polarization in Yeast.  It is based on the model published in Altschuler 2008.  The spatial domain for the model is a 3D sphere with the surface of the sphere reperesnting the membrane of the yeast cell and the volume of the sphere reperesenting the cytoplasm.  Cytoplasmic Cdc42 (species "A") can spontantously associate with the membrate, becoming membrane bound (Species "B").  Membrance bound Cdc42 can diffuse only on the membrane, but can spontaneously disaccociate, becoming "A" again.  "B" on the membrane act to recruit more "A" to the membrane. 

In [1]:
import os
import pyurdme
import dolfin
import mshr
import numpy

In [2]:
class Membrane(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

In [3]:
class YeastPolarization(pyurdme.URDMEModel):
    def __init__(self, Nval=1000, model_name="YeastPolarization"):
        pyurdme.URDMEModel.__init__(self, model_name)
        # Define Parameters
        k_on = pyurdme.Parameter(name="k_on", expression=0.0001/60)
        k_off = pyurdme.Parameter(name="k_off", expression=9.0/60)
        k_fb = pyurdme.Parameter(name="k_fb", expression=10.0/60)
        self.add_parameter([k_on, k_off, k_fb])
        # Define Species
        A = pyurdme.Species(name="A", diffusion_constant=10)
        B = pyurdme.Species(name="B", diffusion_constant=0.0053)
        self.add_species([A, B])
        #Define Geometry
        sphere = mshr.Sphere(dolfin.Point(0.0, 0.0, 0.0), 5.0)
        self.mesh = pyurdme.URDMEMesh(mesh=mshr.generate_mesh(sphere, 14))
        # Define Subdomains
        self.add_subdomain(Membrane(), 2)
        # Define Reactions
        R1 = pyurdme.Reaction(reactants={A:1}, products={B:1}, rate=k_on,  restrict_to=2)
        R2 = pyurdme.Reaction(reactants={B:1}, products={A:1}, rate=k_off, restrict_to=2)
        R3 = pyurdme.Reaction(reactants={A:1, B:1}, products={B:2}, rate=k_fb)
        self.add_reaction([R1, R2, R3])
        #Define initial populations
        A_initial = int(0.9*Nval)
        B_initial = Nval - A_initial
        self.set_initial_condition_scatter({A:A_initial},[1])
        self.set_initial_condition_scatter({B:B_initial},[2])
        # Define simulation timespan
        self.timespan(range(10000))

In [4]:
model = YeastPolarization()
%time result = model.run()

CPU times: user 1.25 s, sys: 65.2 ms, total: 1.31 s
Wall time: 7.2 s


In [5]:
result.display("B",999)

In [6]:
model.mesh

# Parameter Sweep

## WARNING: This can take a long time to complete

In [ ]:
#This currently uses the voxels listed in voxels_center (which are the top "X" percent of particle counts in B_vals) and checks
#an area around the center equivalent to roughly 10% of the membrane surface area and calculates a percentage of membrane particles
#found for each one
def percent_polarized_v_time(result):
    import numpy
    import time
    tic = time.time()
    model = result.model
    data = model.get_solver_datastructure()
    sd = model.get_subdomain_vector()
    mesh_coordinates = model.mesh.coordinates()

    vol = data['vol']
    membrane_voxels = numpy.where(sd==2)[0]
    
    #for voxel_index, voxel_coords in enumerate(model.mesh.coordinates()):
    #    if sd[voxel_index] == 2:
    #        membrane_voxels.append(voxel_index)
    
    B_vals_list = []
    B_vals_to_check_list = []
    min_to_check_list = []
    voxels_center_list = []
    B_vals_sort_list = []
    
    B_vals_plot = result.get_species('B', concentration=False)
    B_vs_time = numpy.sum(B_vals_plot,axis=1)
    
    for n,t in enumerate(model.tspan):
        B_val_temp = B_vals_plot[n,:]#result.get_species('B', timepoints=n, concentration=False)
        B_vals_list.append(B_val_temp)
        B_vals_sort_list.append(sorted(B_vals_list[n]))

    for idx,val in enumerate(B_vals_sort_list):
        B_vals_to_check_list.append(B_vals_sort_list[idx][int(len(B_vals_sort_list[idx]) * .98) : int(len(B_vals_sort_list[idx]) * 1.0)]) 
        min_to_check_list.append(min(B_vals_to_check_list[idx]))
        voxels_center_list.append(map(int,[i for i, j in enumerate(B_vals_list[idx]) if j >= min_to_check_list[idx] and sd[i] == 2])) 
    
    #print "setup {0}".format(time.time()-tic)
    percent_polarized_max_list = []
    for n,t in enumerate(model.tspan):
        
        B_check = sum(B_vals_list[n])
        particle_sum_list = []
        percentage_around_center_list = []
        voxel_counter_list = []
        voxels_visited_list = []
        if B_check==0:
            percentage_around_center = 0
            percentage_around_center_list.append([percentage_around_center])
        else:
            j = 0
            for v in voxels_center_list[n]:
                particle_sum = 0.0
                percentage_around_center = 0.0
                voxel_counter = 0.0
                voxels_visited = []
                #x_center = model.mesh.coordinates()[v,0]
                #y_center = model.mesh.coordinates()[v,1]
                #z_center = model.mesh.coordinates()[v,2]
                x_center, y_center, z_center = mesh_coordinates[v,:]
                dist_center = numpy.sqrt(x_center*x_center + y_center*y_center + z_center*z_center)
                for i in membrane_voxels:
                    #x_check = model.mesh.coordinates()[i,0]
                    #y_check = model.mesh.coordinates()[i,1]
                    #z_check = model.mesh.coordinates()[i,2]
                    try:
                        x_check, y_check, z_check = mesh_coordinates[i,:]
                    except ValueError as e:
                        print mesh_coordinates[i,:]
                        print i
                        print e
                        raise e
                    dist_check = numpy.sqrt(x_check*x_check + y_check*y_check + z_check*z_check)
                    angle_between = numpy.arccos(numpy.dot([x_center,y_center,z_center],[x_check,y_check,z_check])/(dist_check*dist_center) - 1e-8)
                    if angle_between <= numpy.arcsin(.6):
                        particle_sum = particle_sum + B_vals_list[n][i]
                        voxel_counter = voxel_counter + 1
                        voxels_visited.append(i)

                particle_sum_list.append(particle_sum)
                voxel_counter_list.append(voxel_counter)
                voxels_visited_list.append([voxels_visited,v])

                percentage_around_center = particle_sum_list[j]/B_check*100 

                percentage_around_center_list.append([percentage_around_center,v])
                j = j + 1
            
        percent_polarized_max_list.append(max(percentage_around_center_list)[0])
        #print "n={0} {1}s".format(n, time.time()-tic)
        
    return [percent_polarized_max_list,numpy.mean(percent_polarized_max_list),numpy.std(percent_polarized_max_list),B_vs_time]

In [7]:
from molnsutil import ParameterSweep

In [18]:
Nval_list = [200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,3000,4000,5000,7500,10000,20000,40000]
print "Parameter sweep of {0} values".format(len(Nval_list))
sweep = ParameterSweep(model_class=YeastPolarization, parameters={'Nval':Nval_list})

Parameter sweep of 34 values


In [ ]:
%time ret = sweep.run(mapper=percent_polarized_v_time,number_of_trajectories=1,store_realizations=False)

In [ ]:
#Collect data from all result objects
N_list = []
mean_over_time_vs_N = []
std_over_time_vs_N = []
B_vs_time_list = []
percent_polarized_vs_time_list = []
length = len(ret)
for i in range(0,len(ret)):
    percent_polarized_vs_time_list.append(ret[i].result[0][0])
    mean_over_time_vs_N.append(ret[i].result[0][1])
    std_over_time_vs_N.append(ret[i].result[0][2])
    B_vs_time_list.append(ret[i].result[0][3])
    N_list.append(ret[i].parameters['Nval'])

In [ ]:
fig = plt.figure()
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.8])
plt.errorbar(N_list, mean_over_time_vs_N, std_over_time_vs_N, linestyle='None', marker='^')
pylab.xlim([100,4100])
pylab.ylim([-20,110])
plt.axvline(x=471, ymin=-20, ymax = 110, linestyle = '--',color='k')
plt.ylabel('Polarization Percentage',fontsize=15)
plt.xlabel('Number of Cdc42 Molecules',fontsize=15)
fig = plt.gcf()
fig.set_dpi(500)
fig.set_size_inches(8.5,5.5)
plt.tick_params(axis='both', which='major', labelsize=15)

ax2 = fig.add_axes([0.55, 0.55, 0.3, 0.3])
ax2.set_xticks([10000,20000,30000,40000])
plt.errorbar(N_list, mean_over_time_vs_N, std_over_time_vs_N, linestyle='None', marker='^')
plt.plot([0,40000],[10,10],'k--')
pylab.xlim([100,41000])
pylab.ylim([-20,110])
#plt.axvline(x=471, ymin=-20, ymax = 110, linestyle = '--',color='k')
plt.ylabel('Polarization Percentage',fontsize=10)
plt.xlabel('Number of Cdc42 Molecules',fontsize=10)
fig = plt.gcf()
fig.set_dpi(10000)
fig.set_size_inches(8.5,5.5)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.show()

In [ ]:
def yyplot(N=0):
    fifty = [50,50]
    fig, ax1 = pyplot.subplots()
    ax1.plot(B_vs_time_list[N], 'b-')
    ax1.set_xlabel('Time [s]',fontsize=15)
    # Make the y-axis label and tick labels match the line color.
    ax1.set_ylabel('# Cdc42 on Membrane', fontsize=16, color='b')
    for tl in ax1.get_xticklabels():
        tl.set_fontsize(15)
    for tl in ax1.get_yticklabels():
        tl.set_color('b')
        tl.set_fontsize(15)


    ax2 = ax1.twinx()
    ax2.plot(percent_polarized_vs_time_list[N], 'r-',[0,10000],fifty,'r--')
    ax2.set_ylabel('Percent Polarization', fontsize=16, color='r')
    ax2.xticks([20,40,60,80,100],[20,40,60,80,100])
    for tl in ax2.get_yticklabels():
        tl.set_color('r')
        tl.set_fontsize(15)
    ax2.text(4000,15,"N={0}".format(N_list[N]),fontsize=18,fontweight='bold')
    plt.show()

In [ ]:
yyplot(N=1)

In [ ]:
yyplot(N=8)

In [ ]:
yyplot(N=16)

In [ ]:
yyplot(N=21)